## Regresiones varias + polinómica

In [1]:
# Cargo el módulo de numpy
#-------------------------
import numpy as np
import matplotlib.pyplot as plt
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
%matplotlib  

Using matplotlib backend: Qt5Agg


In [34]:
# Definición de las clases
#=========================

# Definición de la clase para levantar (y dividir) los datos
#===========================================================
class Data(object):

    def __init__(self, path):
        self.dataset = self._build_dataset(path)

    def _build_dataset(self, path):
        # Armo una estructura de datos para guardarlos ahí
        #-------------------------------------------------
        structure = [('income', np.float),
                     ('happiness', np.float)]
        
        # Abro el archivo lo recorro llenando la estructura creada línea a línea
        #-----------------------------------------------------------------------
        with open(path, encoding="utf8") as data_csv:

            data_gen = ((float(line.split(',')[1]), float(line.split(',')[2])) # add here + 10 in second value
                        for i, line in enumerate(data_csv) if i != 0)
            embeddings = np.fromiter(data_gen, structure)

        return embeddings
    
    # Separo los los datos (train y test)
    #------------------------------------
    def split(self, percentage): # 0.8
        X = self.dataset['income']
        y = self.dataset['happiness']

        permuted_idxs = np.random.permutation(X.shape[0])

        train_idxs = permuted_idxs[0:int(percentage * X.shape[0])]

        test_idxs = permuted_idxs[int(percentage * X.shape[0]): X.shape[0]]

        X_train = X[train_idxs]
        X_test = X[test_idxs]

        y_train = y[train_idxs]
        y_test = y[test_idxs]

        return X_train, X_test, y_train, y_test

# Clase base de la que heredan las que vayamos implementando
#-----------------------------------------------------------
# Es conveniente tener una clase base de la que vayan heredando las demás. Siempre habrá un método fit
# y un método predict. Pero en esta clase base puede haber definiciones de atributos comunes a todas
#===========================================================
class BaseModel(object):

    def __init__(self):
        self.model = None

    def fit(self, X, Y):
        return NotImplemented

    def predict(self, X):
        return NotImplemented


class ConstantModel(BaseModel):
    # El modelo constante solo saca la media de los datos y devuelve ese valor
    # Es útil para comparar. Ningún modelo debería ser peor que este.
    #-------------------------------------------------------------------------
    def fit(self, X, Y):
        W = Y.mean()
        self.model = W

    def predict(self, X):
        # La "predicción" consiste en devolver la media para todos los valores
        return np.ones(len(X)) * self.model

# Modelo de la regresión lineal
#==============================
class LinearRegression(BaseModel):
    # Este modelo de regresión lineal ajusta únicamente la pendiente, no contempla la ordenada al origen
    def fit(self, X, y):
        # Verificamos si X es un vector o una matriz
        if len(X.shape) == 1:
            # Esta es una manera de escribir la pseudo-inversa (X'.X)^(-1).X'.y
            W = X.T.dot(y) / X.T.dot(X)
        else:
            # Y esta es la manera con matrices
            W = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.model = W

    def predict(self, X):
        return self.model * X
    
# Modelo que incluye la ordenada al origen (b)
# ============================================
class LinearRegressionWithB(BaseModel):

    def fit(self, X, y):
        # En el caso de ajustar con ordenada al origen le agregamos la columna de b con unos
        # (Le agrega la fila abajo y luego traspongo --> Vectores columna)
        X_expanded = np.vstack((X, np.ones(len(X)))).T
        W = np.linalg.inv(X_expanded.T.dot(X_expanded)).dot(X_expanded.T).dot(y)
        self.model = W

    def predict(self, X):
        X_expanded = np.vstack((X, np.ones(len(X)))).T
        return X_expanded.dot(self.model)

# Modelo de la regresión cuadrática
#==================================
class QuadraticRegression(BaseModel):

    def fit(self, X, y):
        # Armamos la matriz de ajuste
        X_expanded = np.vstack((X**2, X, np.ones(len(X)))).T
        W = np.linalg.inv(X_expanded.T.dot(X_expanded)).dot(X_expanded.T).dot(y)
        
        self.model = W

    def predict(self, X):
        X_expanded = np.vstack((X**2, X, np.ones(len(X)))).T
        return X_expanded.dot(self.model)

# Modelo de la regresión cuadrática
#==================================
class PolyRegression(BaseModel):
    
    def fit(self, X, y, n):
        # Tomo X y le agrego el término independiente
        X_expanded = np.vstack((X, np.ones(len(X))))
        for i in range(n-1):
            # Armamos la matriz de ajuste a partir del grado del polinomio
            X_n = X**(n-i)
            X_expanded = np.vstack((X_n, X_expanded))
         
        X_expanded = X_expanded.T
            
        W = np.linalg.inv(X_expanded.T.dot(X_expanded)).dot(X_expanded.T).dot(y)
        
        self.model = W

    def predict(self, X, n):
        # Tomo X y le agrego el término independiente
        X_expanded = np.vstack((X, np.ones(len(X))))
        for i in range(n-1):
            # Armamos la matriz de ajuste a partir del grado del polinomio
            X_n = X**(n-i)
            X_expanded = np.vstack((X_n, X_expanded))
         
        X_expanded = X_expanded.T
        
        return X_expanded.dot(self.model)
    
""" class GradientDescent(BaseModel)

    def gradient_descent(X_train, y_train, lr=0.01, amt_epochs=100):
        n = X_train.shape[0]
        m = X_train.shape[1]
        print('X.shape:{}x{}\n'.format(n,m))
        
        # Inicializamos los pesos
        W=np.random.randn(m).reshape(m,1)
        for idx=1 to amt_epochs
            # Calculo la estimación
            y_hat=X_train*W
            # Calculo el error
            e=y_train-y_hat
            # Calculo el gradiente
            grad_sum = np.sum(error*X_train,axis=0)
            grad_mul =...
            # Actualizo el valor
            
"""
    
# Clases de métricas
#===================
class Metric(object):
    def __call__(self, target, prediction):
        return NotImplemented

# Por ahora solo esta --> Error cuadrático medio
class MSE(Metric):
    def __call__(self, target, prediction):
        n = target.size
        return np.sum((target - prediction) ** 2) / n


In [64]:
# Armamos el main
#----------------
if __name__ == '__main__':
    
    # Llamo al dataset sobre el que voy a trabajar
    #---------------------------------------------
    dataset = Data('../income.data.csv')
    
    # Hacemos la partición del dataset
    #---------------------------------
    X_train, X_test, y_train, y_test = dataset.split(0.8)

    # Llamamos a la regresión lineal (como es un __call__ la llamamos como si fuese función)
    #---------------------------------------------------------------------------------------
    linear_regression = LinearRegression()
    linear_regression.fit(X_train, y_train)
    lr_y_hat = linear_regression.predict(X_test)

    # Llamamos a la regresión lineal con parámetro "b"
    #-------------------------------------------------
    linear_regression_b = LinearRegressionWithB()
    linear_regression_b.fit(X_train, y_train)
    lrb_y_hat = linear_regression_b.predict(X_test)
    
    print('Regresión lineal m:[{}] - b:[{}]'.format(linear_regression_b.model[0],linear_regression_b.model[1]))
    
    # Llamamos a la regresión cuadrática
    #-----------------------------------
    quadratic_regression = QuadraticRegression()
    quadratic_regression.fit(X_train,y_train)         # Ajuste sobre train data
    qrb_y_hat = quadratic_regression.predict(X_test)  # Error sobre test data
    
    print('Regresión cuadrática a:[{}] - b:[{}] - c:[{}]'.format(quadratic_regression.model[0],quadratic_regression.model[1],quadratic_regression.model[2]))
    
    # Llamamos a la regresión polinómica
    #-----------------------------------
    grado = 3
    poly_regression = PolyRegression()
    poly_regression.fit(X_train,y_train,grado)          # Ajuste sobre train data
    poly_y_hat = poly_regression.predict(X_test,grado)  # Error sobre test data
    
    print('Regresión cuadrática a:[{}] - b:[{}] - c:[{}]'.format(quadratic_regression.model[0],quadratic_regression.model[1],quadratic_regression.model[2]))
    
    # Hacemos el ajuste contra el modelo constante
    #---------------------------------------------
    constant_model = ConstantModel()
    constant_model.fit(X_train, y_train)
    ct_y_hat = constant_model.predict(X_test)

    mse = MSE()
    lr_mse = mse(y_test, lr_y_hat)
    lrb_mse = mse(y_test, lrb_y_hat)
    qrb_mse = mse(y_test, qrb_y_hat)
    poly_mse = mse(y_test, poly_y_hat)
    ct_mse = mse(y_test, ct_y_hat)

    # Dibujamos los resultados del ajuste
    #------------------------------------
    x_plot = np.linspace(0, 10, 10)
    lr_y_plot = linear_regression.model * x_plot # m*x
    lrb_y_plot = linear_regression_b.model[0] * x_plot + linear_regression_b.model[1]  # m*x + b
    qrb_y_plot = quadratic_regression.model[0] * (x_plot**2) + quadratic_regression.model[1] * x_plot + quadratic_regression.model[2]
    
    poly_y_plot=poly_regression.model[grado]
    #print('poly {}:'.format(poly_regression.model))
    #print('quad {}:'.format(quadratic_regression.model))
    for i in range(grado):
        #print(i)
        #print(grado-i)
        poly_y_plot = poly_y_plot + poly_regression.model[i]*(x_plot**(grado-i))
   
    plt.scatter(X_train, y_train, color='b', label='dataset')
    plt.plot(x_plot, lr_y_plot, color='m', label=f'LinearRegresion(MSE={lr_mse:.3f})')
    plt.plot(x_plot, lrb_y_plot, color='r', label=f'LinearRegresionWithB(MSE={lrb_mse:.3f})')
    plt.plot(x_plot, qrb_y_plot, color='y', label=f'QuadraticRegresion(MSE={qrb_mse:.3f})')
    plt.plot(X_test, ct_y_hat, color='g', label=f'ConstantModel(MSE={ct_mse:.3f})')
    plt.plot(x_plot, poly_y_plot, color='k', label=f'PolyRegresion(MSE={poly_mse:.3f})- Grado:{grado:.0f}')
    plt.legend()
    plt.show()

Regresión lineal m:[0.6947483924490316] - b:[0.29425196920161806]
Regresión cuadrática a:[-0.00467263338155886] - b:[0.736548863771695] - c:[0.214869955949674]
Regresión cuadrática a:[-0.00467263338155886] - b:[0.736548863771695] - c:[0.214869955949674]
poly [ 0.02128776 -0.00192681  0.6300705   0.34389355]:
quad [-0.00467263  0.73654886  0.21486996]:


#### Conslusiones

- No hay practicamente diferencia entre el ajuste lineal (con ordenada al origen) y el cuadrático evaluando el error cuadrátrico medio. Ajustarlo con el modelo cuadrático es hacer un "overfitting"

In [23]:
whos

Variable                Type                     Data/Info
----------------------------------------------------------
BaseModel               type                     <class '__main__.BaseModel'>
ConstantModel           type                     <class '__main__.ConstantModel'>
Data                    type                     <class '__main__.Data'>
LinearRegression        type                     <class '__main__.LinearRegression'>
LinearRegressionWithB   type                     <class '__main__.LinearRegressionWithB'>
MSE                     type                     <class '__main__.MSE'>
Metric                  type                     <class '__main__.Metric'>
PolyRegression          type                     <class '__main__.PolyRegression'>
QuadraticRegression     type                     <class '__main__.QuadraticRegression'>
X_test                  ndarray                  100: 100 elems, type `float64`, 800 bytes
X_train                 ndarray                  398: 398 elems, 